In [ ]:
'''
Citations :
[1] “Beautiful Soup documentation,” Beautiful Soup 4.4.0 documentation. [Online]. Available: https://beautiful-soup-4.readthedocs.io/en/latest/.
[2] "Selenium." [Online]. Available: https://www.selenium.dev/. 
'''

In [ ]:
import time
from html.parser import HTMLParser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from bs4 import BeautifulSoup
from re import sub
from sys import stderr
from traceback import print_exc
import re
import numpy as np

In [ ]:
opi_df =  pd.read_csv('final_opioid_keywords.csv')

In [ ]:
twit= []
for opis in opi_df['Generic_Name']:
    browser = webdriver.Chrome('driver/chromedriver')
    browser.get("https://twitter.com/search?q="+opis+"%20side%20effects&src=recent_search_click")
    time.sleep(1)

    # elem = browser.find_element_by_tag_name("body")

    no_of_pagedowns = 0
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        no_of_pagedowns+=1
        print(no_of_pagedowns)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(4)
        soup = BeautifulSoup(browser.page_source)
        twit.append([opis,soup])
        new_height = browser.execute_script("return document.body.scrollHeight")
        # break condition
        if new_height == last_height:
            break
        last_height = new_height

    browser.quit()

In [ ]:
cmntsOpi = pd.DataFrame(twit,columns=['Components','Comments'])
len(cmntsOpi['Comments'])

In [ ]:
twisS = []
for twis in range(len(cmntsOpi)):    
    htmlParse = BeautifulSoup(cmntsOpi.Comments[twis], 'html.parser')
    for hm in htmlParse.find_all('div',class_='css-1dbjc4n r-1igl3o0 r-qklmqi r-1adg3ll r-1ny4l3l'):
        twisS.append([hm.find('div',class_="css-1dbjc4n").text,hm.find('time').text,cmntsOpi.Components[twis]])

In [ ]:
newCmntsOpi = pd.DataFrame(twisS,columns=['Comments','DateTime','Components'])
newCmntsOpi['Comments'] = [re.sub(r"\[.*\n?","",str(x),flags=re.MULTILINE) for x in newCmntsOpi['Comments']]
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].str.replace(r'<[^<>]*>', '', regex=True)
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].str.split('\n\n')
newCmntsOpi = newCmntsOpi.explode('Comments')
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].replace(r'^\s*$', np.nan, regex=True)
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].str.rstrip()
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].str.lstrip()
newCmntsOpi = newCmntsOpi.dropna()
newCmntsOpi = newCmntsOpi.reset_index(drop=True)
len(newCmntsOpi)

In [ ]:
newCmntsOpi.to_csv('webdata/twitter_effects.csv')
newCmntsOpi